# 昇腾PyTorch实战：bert-base-chinese应用

## 实验简介

bert-base-chinese是一种基于 BERT架构预训练的语言模型，它包含了 12 层 Transformer 编码器和 110 万个参数，可以用于各种中文自然语言处理任务，如文本分类、命名实体识别、情感分析等。使用 `bert-base-chinese` 模型时，可以将其作为一个特征提取器，将输入的文本转换为固定长度的向量表示，然后将这些向量输入到其他机器学习模型中进行训练或推断。也可以对 `bert-base-chinese` 进行微调，将其用于特定任务的训练。
本实验将简单使用bert-base-chinese 对文本进行向量转换以及掩码预测。

## 实验目的

- 了解bert-base-chinese的使用方法；

## 背景介绍 

它使用中文维基百科数据进行训练，通过掩码语言模型和下一句预测两个预训练任务学习语言表示。该模型具有12层Transformer编码器，能够捕捉文本中的复杂关系。其目的是提供一个强大的基础模型，可通过微调应用于多种中文自然语言处理任务，如文本分类、命名实体识别和问答系统等。这一模型的推出显著提升了中文NLP应用的效果。

## 实验环境准备：
### 1. 模型下载
使用wget方式进行模型下载，命令如下:

In [1]:
!wget --no-check-certificate https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_models/1bd74bd4f97511ef87f9fa163edcddae/bert-base-chinese.zip

--2025-06-13 18:22:00--  https://model-community-picture.obs.cn-north-4.myhuaweicloud.com/ascend-zone/notebook_models/1bd74bd4f97511ef87f9fa163edcddae/bert-base-chinese.zip
Resolving model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)... 121.36.121.227, 121.36.121.226
Connecting to model-community-picture.obs.cn-north-4.myhuaweicloud.com (model-community-picture.obs.cn-north-4.myhuaweicloud.com)|121.36.121.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3173464872 (3.0G) [application/zip]
Saving to: 'bert-base-chinese.zip'

bert-base-chinese.z 100%[===================>]   2.96G  36.9MB/s    in 88s     

2025-06-13 18:23:29 (34.5 MB/s) - 'bert-base-chinese.zip' saved [3173464872/3173464872]



In [2]:
!unzip bert-base-chinese.zip

Archive:  bert-base-chinese.zip
   creating: bert-base-chinese/
   creating: bert-base-chinese/.git/
   creating: bert-base-chinese/.git/info/
  inflating: bert-base-chinese/.git/info/exclude  
  inflating: bert-base-chinese/.git/index  
   creating: bert-base-chinese/.git/refs/
   creating: bert-base-chinese/.git/refs/tags/
   creating: bert-base-chinese/.git/refs/remotes/
   creating: bert-base-chinese/.git/refs/remotes/origin/
 extracting: bert-base-chinese/.git/refs/remotes/origin/HEAD  
   creating: bert-base-chinese/.git/refs/heads/
  inflating: bert-base-chinese/.git/refs/heads/main  
  inflating: bert-base-chinese/.git/packed-refs  
   creating: bert-base-chinese/.git/objects/
   creating: bert-base-chinese/.git/objects/dd/
  inflating: bert-base-chinese/.git/objects/dd/9401af0a4713df0e75066ae78d5bf6fa6c7116  
   creating: bert-base-chinese/.git/objects/info/
   creating: bert-base-chinese/.git/objects/85/
 extracting: bert-base-chinese/.git/objects/85/85f10750f4d5a9d975775ee81

### 2. 环境配置

#### 2.1. 驱动、固件以及CANN环境配置

在本实验之前，我们需要确保已经正确配置昇腾环境，如果尚未安装配套的昇腾NPU驱动、固件以及CANN开发套件包，请参考[昇腾官网](https://www.hiascend.com/document/detail/zh/canncommercial/80RC2/softwareinst/instg/instg_0003.html?Mode=PmIns&OS=Ubuntu&Software=cannToolKit)进行安装。


#### 2.2. 安装所需包

In [3]:
%pip install transformers==4.37.2 
%pip install torch==2.1.0

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 34.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.1
    Uninstalling transformers-4.37.1:
      Successfully uninstalled transformers-4.37.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openmind-accelerate 0.5.2 requires accelerate==0.28.0, but you have accelerate 0.26.1 which is incompatible.
openmind-accelerate 0.5.2 requires torch-npu==2.1.0.post3, but you have torch-npu 2.1.0.post6 which is incompatible.
openmind-accelerate 0.5.2 requires transformers==4.39.2, but yo

#### 2.3. npu环境配置

In [4]:
%pip install torch_npu==2.1.0.post6
%pip install numpy==1.24.4  
%pip install pyyaml
%pip install attrs
%pip install scipy
%pip install ml-dtypes
%pip install --upgrade jupyter ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.huaweicloud.com/repository/pypi/simple

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defa

## 实验步骤：
### 1. 导入所用包

In [6]:
import torch
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModelForMaskedLM

### 2. 加载模型

使用前面步骤`git clone`的模型进行加载

In [7]:
model_path = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)

### 3. bert-base-chinese模型使用
#### 3.1 使用bert-base-chinese模型对文本进行编码

In [8]:
def encode_text_with_bert(text):
    encoded_text = tokenizer.encode(text)[1: -1]
    encoded_tensor = torch.LongTensor([encoded_text])

    with torch.no_grad():
        output = model(encoded_tensor)
    return output[0]

# 结果展示
text1 = "你好，美丽中国"
result = encode_text_with_bert(text1)
print('text1编码的形状:', result.size())
print('text1编码:\n', result)

text1编码的形状: torch.Size([1, 7, 768])
text1编码:
 tensor([[[ 0.0781, -0.7386, -0.5120,  ...,  1.0695, -0.4252, -0.3970],
         [ 0.3118, -0.2283, -0.2513,  ..., -0.0618,  0.8715, -0.0833],
         [ 0.0287, -0.4937, -0.5554,  ...,  0.1643,  0.8771,  0.0018],
         ...,
         [-0.3068, -0.3406,  0.0525,  ...,  0.5506,  0.8915, -0.3713],
         [-0.1079, -0.0951, -0.1549,  ...,  0.8432,  0.7255, -0.5235],
         [-0.0414, -0.3786,  0.1590,  ...,  0.3844,  0.7464, -0.4266]]])


#### 3.2 使用bert-base-chinese模型对文本进行填充

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)
text = "生活的真谛是[MASK]。"
input_ids = tokenizer.encode(text, return_tensors='pt')

# 创建掩码
masked_index = 7
input_ids[:, masked_index] = tokenizer.mask_token_id

# 进行预测
outputs = model(input_ids)
logits = outputs.logits

# 将预测结果转换为文本
predicted_text = tokenizer.convert_ids_to_tokens(logits[:, masked_index].argmax(dim=1).cpu())

# 打印结果
print(f"Original: {text}")
print(f"Predicted: {predicted_text[0]}")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Original: 生活的真谛是[MASK]。
Predicted: 美
